In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import ViTFeatureExtractor, TFViTForImageClassification
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import keras_tuner as kt
from sklearn.metrics import (accuracy_score, roc_curve, auc, precision_recall_curve, 
                             confusion_matrix, roc_auc_score, RocCurveDisplay, PrecisionRecallDisplay, 
                             classification_report, recall_score, balanced_accuracy_score, f1_score, precision_score)
from sklearn.inspection import permutation_importance
import seaborn as sns
import shap
from sklearn.preprocessing import label_binarize 
import cv2

In [ ]:
dataset_path = r"D:\Download\archive\seg_train\seg_train"

In [ ]:
img = cv2.imread(r"D:\Download\archive\seg_test\seg_test\buildings\20061.jpg")
print(np.min(img), np.max(img))

In [ ]:
img_size = (224, 224)
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

In [ ]:
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
num_classes = train_generator.num_classes

In [ ]:
model_name = "google/vit-base-patch16-224-in21k"

In [ ]:
best_model = TFViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_classes
)

In [ ]:
best_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
best_model.summary()

In [ ]:
history = best_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
y_predict = best_model.predict(val_generator)
y_test = val_generator.classes
y_pred = np.argmax(y_predict, axis = 1)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)

plt.matshow(matrix)
plt.colorbar()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
sns.heatmap(matrix, square  =True, annot = True, cbar = False)
plt.xlabel('predicted value')
plt.ylabel('true value')

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class='ovr')
print(roc_auc_ovr)
classes = np.unique(y_test)

y_test_bin = label_binarize(y_test, classes = classes)
n_classes = y_test_bin.shape[1]

for i in range(n_classes):
    RocCurveDisplay.from_predictions(
        y_test_bin[:,i],
        y_pred[:,i],
        name=f"Class {i} vs Rest",
        plot_chance_level =(i==0)
    )

In [ ]:
precision = dict()
recall = dict()
pr_auc = dict()

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:,i], y_pred[:,i])
    pr_auc[i] = auc(recall[i], precision[i])
    PrecisionRecallDisplay.from_predictions(
        y_test_bin[:,i],
        y_pred[:,i],
        name=f"Class {i} vs Rest",
        plot_chance_level =(i==0)
    )

plt.show()

In [ ]:
print("\nModel Evaluation:")
results = best_model.evaluate(val_generator, y_test, verbose=0)
metrics = dict(zip(best_model.metrics_names, results))
for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

In [ ]:
y_train_pred = best_model.predict(train_generator)
y_test_pred = best_model.predict(val_generator)

if y_train.ndim > 1 and y_train.shape[1] > 1:
    y_train_true = np.argmax(y_train, axis=1)
    y_test_true = np.argmax(y_test, axis=1)
    y_train_pred = np.max(y_train_pred, axis=1)  
    y_test_pred = np.max(y_test_pred, axis=1)
else:
    y_train_true = y_train.ravel() 
    y_test_true = y_test.ravel()

train_auc = roc_auc_score(y_train_true, y_train_pred, multi_class='ovr')
test_auc = roc_auc_score(y_test_true, y_test_pred, multi_class='ovr')

train_loss = history.history['loss']
test_loss = history.history['val_loss'] 
train_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_acc, label='Train Accuracy', linestyle='dashed')
plt.plot(test_acc, label='Test Accuracy', linestyle='dashed')
plt.axhline(train_auc, color='blue', linestyle='solid', label=f'Train AUC: {train_auc:.3f}')
plt.axhline(test_auc, color='red', linestyle='solid', label=f'Test AUC: {test_auc:.3f}')
plt.title('Training & Test Accuracy with AUC')
plt.ylabel('Accuracy / AUC')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(test_loss, label='Test Loss')
plt.title('Training and Test Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
balanced_acc = balanced_accuracy_score(y_test, y_pred)

print("\nAdvanced Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Balanced Accuracy: {balanced_acc:.4f}")

In [ ]:
print("\nAdvanced Metrics:")
print(f"- Maximum Training Accuracy: {max(train_acc):.4f}")
print(f"- Minimum Test Loss: {min(test_loss):.4f}")
print(f"- Optimal Epochs: {len(train_loss)}")
print(f"- Maximum Training AUC: {train_auc:.4f}")
print(f"- Test AUC: {test_auc:.4f}")

In [ ]:
explainer = shap.Explainer(best_model, train_generator)

if hasattr(val_generator, 'numpy'):
    val_generator = val_generator.numpy()
shap_values = explainer(val_generator)

In [ ]:
shap.summary_plot(shap_values.values, val_generator)

In [ ]:
def predict_image(model, image_path, class_indices, img_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size) / 255.0
    img = np.expand_dims(img, axis=0)
    
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    class_label = list(class_indices.keys())[predicted_class] 
    
    plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Predicted: {class_label}")
    plt.show()

predict_image(vit, "path_to_new_image.jpg", train_generator.class_indices)